In [1]:
#We are using kora to install rdkit library
!pip install kora
import kora.install.rdkit

In [2]:
#importing all the necessary libraries
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from google.colab import files
import numpy as np
import pandas as pd
import io
import requests
import json 
import os

In [3]:
 def mol_data(smiles):

   smiles['mol'] = smiles['CanonicalSMILES'].apply(lambda x: Chem.MolFromSmiles(x)) 
   smiles['mol_w'] = smiles['mol'].apply(lambda x: Descriptors.ExactMolWt(x))
   smiles['XLogP'] = smiles['mol'].apply(lambda x: Descriptors.MolLogP(x))
   smiles['NumValenceElectrons'] = smiles['mol'].apply(lambda x: Descriptors.NumValenceElectrons(x))              
   smiles['NumAromaticRings'] = smiles['mol'].apply(lambda x: Descriptors.NumAromaticRings(x))        
   smiles['NumSaturatedRings'] = smiles['mol'].apply(lambda x: Descriptors.NumSaturatedRings(x))        
   smiles['NumAliphaticRings'] = smiles['mol'].apply(lambda x: Descriptors.NumAliphaticRings(x))
   smiles['RingCount'] = smiles['mol'].apply(lambda x: Descriptors.RingCount(x))        
   smiles['TPSA'] = smiles['mol'].apply(lambda x: Descriptors.TPSA(x))
   smiles['mol'] = smiles['mol'].apply(lambda x: Chem.AddHs(x))
   smiles['num_of_atoms'] = smiles['mol'].apply(lambda x: x.GetNumAtoms())
   smiles['num_of_heavy_atoms'] = smiles['mol'].apply(lambda x: x.GetNumHeavyAtoms())
   c_patt = Chem.MolFromSmiles('C')
   def number_of_atoms(atom_list, data):
       for i in atom_list:
           data['num_of_{}_atoms'.format(i)] = data['mol'].apply(lambda x: len(x.GetSubstructMatches(Chem.MolFromSmiles(i))))

   number_of_atoms(['C','O', 'N', 'Cl'], smiles)
   return 0

In [5]:
# upload main_data.csv, can be downloaded from github(https://github.com/mohitkush/DDH-2020-T2PS07.git)
uploaded = files.upload()
model_data = pd.read_csv(io.BytesIO(uploaded['main_data.csv']))
mol_data(model_data)

Saving main_data.csv to main_data.csv


0

In [9]:
# upload test_drugs.csv, can be downloaded from github(https://github.com/mohitkush/DDH-2020-T2PS07.git)
uploaded2 = files.upload()
test_data = pd.read_csv(io.BytesIO(uploaded2['test_drugs.csv']))
mol_data(test_data)

Saving test_drugs.csv to test_drugs (1).csv


0

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
X=model_data[['XLogP', 'Charge',	'HBondDonorCount',	'HBondAcceptorCount',	'RotatableBondCount', 'mol_w',	'num_of_atoms',	'num_of_C_atoms',	'num_of_O_atoms',	'num_of_N_atoms',	'num_of_Cl_atoms', 'NumValenceElectrons', 'TPSA']]
y=model_data['Phase']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.3434343434343434


In [22]:
prediction = clf.predict(test_data[['XLogP', 'Charge',	'HBondDonorCount',	'HBondAcceptorCount',	'RotatableBondCount', 'mol_w',	'num_of_atoms',	'num_of_C_atoms',	'num_of_O_atoms',	'num_of_N_atoms',	'num_of_Cl_atoms', 'NumValenceElectrons', 'TPSA']])

In [31]:
i=0
lisy = []
for x in prediction:
  i+=1
  if x==4:
    lisy.append(i)

In [36]:
test_data.iloc[lisy].to_csv('Phase4.csv')
files.download('Phase4.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>